# Importando as bibliotecas

In [1]:
import pandas as pd
from db import connection_db as conndb
from db import filters
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.spatial.distance import cdist
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
from pymannkendall import original_test

tqdm.pandas()

# Conectando ao banco de dados do MongoDB

Função para conectar ao MongoDB

In [ ]:
def conectar_mongodb(uri, db_name, collection_name):
    mongo_connection = conndb.MongoDBConnection(uri=uri, database_name=db_name, collection_name=collection_name)
    mongo_connection.connect()
    return mongo_connection.collection

In [ ]:
# Definindo os parâmetros de conexão
uri = 'mongodb://localhost:27017/'
db_name = 'dadosVivamente'
collection_name = 'dadosSemFiltros'
collection_name_filtered = 'posts7anos1anos'

collection = conectar_mongodb(uri, db_name, collection_name)
collection_filtrada = conectar_mongodb(uri, db_name, collection_name_filtered)

# Aplicando pipeline para preparação dos dados

Função para aplicar os pipelines de filtro

In [ ]:
def aplicar_pipelines(collection, collection_filtrada):
    collection_filters = filters.CollectionFilters(collection)
    collection_filters_likes = filters.CollectionFilters(collection_filtrada)

    # Aplicar pipelines
    collection_filters.apply_pipeline1('dadosComFiltrosIniciais')
    collection_filters.apply_pipeline2(7, 1, 'posts7anos1anos')
    collection_filters.apply_pipeline3('postsComBDIAndInfos')
    collection_filters.apply_pipeline4('postsComBDIAndInfosFiltroDataPosts')
    collection_filters_likes.apply_pipeline6('likes')

    return collection_filters.collection, collection_filters_likes.collection

In [ ]:
# Aplicando os pipelinesc
collection, collection_filtrada = aplicar_pipelines(collection, collection_filtrada)

# Carregando os dados em um dataframe

In [ ]:
documentos = collection.find()
df_original = pd.DataFrame(list(documentos))

In [ ]:
# Visualizar as primeiras linhas do dataframe
df_original.head()

In [ ]:
documentos_likes = collection_filtrada.find()
df_likes = pd.DataFrame(list(documentos_likes))

In [ ]:
# Visualizar as primeiras linhas do dataframe
df_likes.head()

In [ ]:
# Transformando a coluna sexo em binária
df_original['sexo'] = df_original['sexo'].map({'F': 0, 'M': 1})

# Removendo colunas desnecessárias
df_original.drop(columns=['_id', 'diaDaSemana'], inplace=True)

# Lista de colunas que precisam ser convertidas
colunas_para_converter = [
    'pessimismo', 'tristeza', 'fracasso', 'prazer', 'culpa', 'punicao', 'estima', 'critica', 'suicida', 'choro',
    'agitacao', 'interesse', 'indecisao', 'desvalorizacao', 'energia', 'sono', 'irritabilidade', 'apetite',
    'concentracao', 'fadiga', 'int_sexo', 'quantAmigos'
]


# Função para preencher valores nulos e converter o tipo de dado
def preencher_e_converter(df, colunas, valor_preenchimento=0, tipo_dados='int64'):
    df[colunas] = df[colunas].fillna(valor_preenchimento)
    df[colunas] = df[colunas].astype(tipo_dados)
    return df

In [ ]:
# Aplicando a função
df_original = preencher_e_converter(df_original, colunas_para_converter)

In [ ]:
# Função para manipular datas e filtrar posts
def manipular_e_filtrar_posts(df):
    df['data'] = df['postCreatedTime'].dt.date
    df['data'] = pd.to_datetime(df['data'])
    df['mes'] = df['data'].dt.to_period('M')
    df['semana'] = df['data'].dt.to_period('W')

    df_data = df[df['data'].dt.year == 2017]
    df_data = df_data[~((df_data['data'].dt.month == 12) & (df_data['data'].dt.year == 2017))]
    df_data = df_data[df_data['data'].dt.month >= 5]
    df_data = df_data[df_data['suicida'] == 3]

    # Remover posts vazios
    df_data = df_data.dropna(subset=['postMessage', 'postStory'], how='all')
    df_data = df_data[~((df_data['postMessage'].str.strip() == '') & (df_data['postStory'].str.strip() == ''))]

    return df_data

In [ ]:
# Aplicando a função
df_posts = manipular_e_filtrar_posts(df_original)
df_posts.head()

Função para calcular e visualizar a distribuição de postagens por usuário

In [ ]:
# Função para calcular e visualizar a distribuição de postagens por usuário
def calcular_e_visualizar_distribuicao(df, coluna_contagem, label_x, titulo):
    post_counts = df.groupby('id_usuario').size().reset_index(name=coluna_contagem)

    plt.hist(post_counts[coluna_contagem], bins=50)
    plt.xlabel(f'Número de {label_x}')
    plt.ylabel('Quantidade de Usuários')
    plt.title(titulo)
    plt.show()

    return post_counts

In [ ]:
# Aplicando a função
post_counts = calcular_e_visualizar_distribuicao(df_posts, 'post_count', 'Postagens',
                                                 'Distribuição de Postagens por Usuário')

Função para definir os limites e filtrar usuários por atividade

In [ ]:
def filtrar_usuarios_por_atividade(post_counts, coluna_contagem, quantil_superior=0.85, quantil_inferior=0.15):
    # Definir os limites superior e inferior com base nos quantis
    limite_superior = post_counts[coluna_contagem].quantile(quantil_superior)
    limite_inferior = post_counts[coluna_contagem].quantile(quantil_inferior)

    # Filtrar usuários com alta e baixa atividade
    usuarios_alta_ativ = post_counts[post_counts[coluna_contagem] > limite_superior]['id_usuario']
    usuarios_baixa_ativ = post_counts[post_counts[coluna_contagem] < limite_inferior]['id_usuario']

    # Exibir o número de usuários filtrados
    print(f'Número de usuários com alta atividade: {len(usuarios_alta_ativ)}')
    print(f'Número de usuários com baixa atividade: {len(usuarios_baixa_ativ)}')

    return usuarios_alta_ativ, usuarios_baixa_ativ

In [ ]:
# Aplicando a função
usuarios_alta_ativ, usuarios_baixa_ativ = filtrar_usuarios_por_atividade(post_counts, 'post_count')

In [ ]:
df_normal = df_posts[
    ~(df_posts['id_usuario'].isin(usuarios_alta_ativ) | df_posts['id_usuario'].isin(usuarios_baixa_ativ))]

norm_post_count = df_normal.groupby('id_usuario').size().reset_index(name='post_count')

df_alta_ativ = df_posts[df_posts['id_usuario'].isin(usuarios_alta_ativ)]
alt_post_count = df_alta_ativ.groupby('id_usuario').size().reset_index(name='post_count')

df_baixa_ativ = df_posts[df_posts['id_usuario'].isin(usuarios_baixa_ativ)]
baixa_post_count = df_baixa_ativ.groupby('id_usuario').size().reset_index(name='post_count')


Função para plotar a distribuição de postagens por usuário

In [ ]:
def plotar_distribuicao(df, coluna, label_x, titulo):
    plt.hist(df[coluna], bins=50)
    plt.xlabel(f'Número de {label_x}')
    plt.ylabel('Quantidade de Usuários')
    plt.title(titulo)
    plt.show()

In [ ]:
# Aplicando a função        
plotar_distribuicao(norm_post_count, 'post_count', 'Postagens', 'Distribuição de Postagens por Usuário (Normal)')
plotar_distribuicao(alt_post_count, 'post_count', 'Postagens', 'Distribuição de Postagens por Usuário (Alta Atividade)')
plotar_distribuicao(baixa_post_count, 'post_count', 'Postagens',
                    'Distribuição de Postagens por Usuário (Baixa Atividade)')

In [ ]:
# Criar um dataframe para conter as colunas que não mudam e o id_usuario
df_respostas_bdi = df_normal.groupby(['id_usuario']).agg(
    pessimismo=('pessimismo', 'first'),
    tristeza=('tristeza', 'first'),
    fracasso=('fracasso', 'first'),
    prazer=('prazer', 'first'),
    culpa=('culpa', 'first'),
    punicao=('punicao', 'first'),
    estima=('estima', 'first'),
    critica=('critica', 'first'),
    suicida=('suicida', 'first'),
    choro=('choro', 'first'),
    agitacao=('agitacao', 'first'),
    interesse=('interesse', 'first'),
    indecisao=('indecisao', 'first'),
    desvalorizacao=('desvalorizacao', 'first'),
    energia=('energia', 'first'),
    sono=('sono', 'first'),
    irritabilidade=('irritabilidade', 'first'),
    apetite=('apetite', 'first'),
    concentracao=('concentracao', 'first'),
    fadiga=('fadiga', 'first'),
    int_sexo=('int_sexo', 'first'),
).reset_index()

df_respostas_bdi.to_csv('df_respostas_bdi.csv', index=False)

# Visualizar as primeiras linhas do dataframe
df_respostas_bdi.head()

In [ ]:
# Remover a coluna 'id_usuario'
df_media_respostas_bdi = df_respostas_bdi.drop(columns=['id_usuario'])

# Calcular a média de cada resposta do BDI
media_respostas_bdi = df_media_respostas_bdi.mean()

# Visualizar as médias
print(media_respostas_bdi)

# Plotar as médias
plt.figure(figsize=(10, 6))
media_respostas_bdi.plot(kind='bar')
plt.xlabel('Pergunta')
plt.ylabel('Média da Resposta')
plt.title('Média das Respostas do BDI')
plt.show()

In [ ]:
df_normal = df_normal.copy()
df_normal.drop(
    columns=['pessimismo', 'tristeza', 'fracasso', 'prazer', 'culpa', 'punicao', 'estima', 'critica', 'suicida',
             'choro', 'agitacao', 'interesse', 'indecisao', 'desvalorizacao', 'energia', 'sono', 'irritabilidade',
             'apetite', 'concentracao', 'fadiga', 'int_sexo'], inplace=True)

df_alta_ativ = df_alta_ativ.copy()
df_alta_ativ.drop(
    columns=['pessimismo', 'tristeza', 'fracasso', 'prazer', 'culpa', 'punicao', 'estima', 'critica', 'suicida',
             'choro', 'agitacao', 'interesse', 'indecisao', 'desvalorizacao', 'energia', 'sono', 'irritabilidade',
             'apetite', 'concentracao', 'fadiga', 'int_sexo'], inplace=True)

df_baixa_ativ = df_baixa_ativ.copy()
df_baixa_ativ.drop(
    columns=['pessimismo', 'tristeza', 'fracasso', 'prazer', 'culpa', 'punicao', 'estima', 'critica', 'suicida',
             'choro', 'agitacao', 'interesse', 'indecisao', 'desvalorizacao', 'energia', 'sono', 'irritabilidade',
             'apetite', 'concentracao', 'fadiga', 'int_sexo'], inplace=True)

In [ ]:
# Calcular a contagem de postagens por usuário
post_counts_normal = df_normal.groupby('id_usuario').size().reset_index(name='post_count')

# Visualizar a distribuição
plotar_distribuicao(post_counts_normal, 'post_count', 'Postagens', 'Distribuição de Postagens por Usuário')

Função para aplicar clustering K-Means

In [ ]:
def aplicar_kmeans(df, coluna_contagem, n_clusters=3, label_x='Postagens'):
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    df['cluster'] = kmeans.fit_predict(df[[coluna_contagem]])

    sns.histplot(data=df, x=coluna_contagem, hue='cluster', bins=30, kde=True)
    plt.xlabel(f'Número de {label_x}')
    plt.ylabel('Frequência')
    plt.title(f'Distribuição dos Clusters de {label_x} por Usuário')
    plt.show()

    return kmeans, df

In [ ]:
# Aplicando a função
kmeans, post_counts_normal = aplicar_kmeans(post_counts_normal, 'post_count', 3)

Função para calcular o Silhouette Score e a inércia

In [ ]:
post_counts_normal.head()

In [ ]:
def calcular_silhouette_e_inercia(df, kmeans, coluna_contagem):
    silhouette_avg = silhouette_score(df[[coluna_contagem]], kmeans.labels_)
    inercia = kmeans.inertia_

    print(f'Silhouette Score médio: {silhouette_avg}')
    print(f'Inércia do modelo: {inercia}')

In [ ]:
# Aplicando a função
calcular_silhouette_e_inercia(post_counts_normal, kmeans, 'post_count')

Função para calcular a inércia para diferentes números de clusters (Método do Cotovelo)

In [ ]:
def calcular_inercia(df, coluna_contagem, max_clusters=10):
    inercia = []
    K = range(1, max_clusters + 1)
    for k in K:
        kmeans = KMeans(n_clusters=k, random_state=0)
        kmeans.fit(df[[coluna_contagem]])
        inercia.append(kmeans.inertia_)

    plt.plot(K, inercia, 'bx-')
    plt.xlabel('Número de Clusters')
    plt.ylabel('Inércia')
    plt.title('Método do Cotovelo para escolher o número ótimo de clusters')
    plt.show()

In [ ]:
# Aplicando a função
calcular_inercia(post_counts_normal, 'post_count')

Função para calcular as distâncias intra e intercluster

In [ ]:
def calcular_distancias_cluster(df, kmeans, coluna_contagem):
    distancias_intracluster = cdist(df[[coluna_contagem]], kmeans.cluster_centers_[kmeans.labels_], 'euclidean')
    media_intracluster = distancias_intracluster.mean()

    distancias_intercluster = cdist(kmeans.cluster_centers_, kmeans.cluster_centers_, 'euclidean')
    media_intercluster = distancias_intercluster[np.triu_indices_from(distancias_intercluster, 1)].mean()

    print(f'Média das distâncias intracluster: {media_intracluster}')
    print(f'Média das distâncias intercluster: {media_intercluster}')

In [ ]:
# Aplicando a função
calcular_distancias_cluster(post_counts_normal, kmeans, 'post_count')

# Usuarios com alta atividade de postagem

In [ ]:
# Calcular a contagem de postagens por usuário
post_counts_alta = df_alta_ativ.groupby('id_usuario').size().reset_index(name='post_count')

# Visualizar a distribuição
plotar_distribuicao(post_counts_alta, 'post_count', 'Postagens', 'Distribuição de Postagens por Usuário')

In [ ]:
# Aplicando a função
kmeans, post_counts_alta = aplicar_kmeans(post_counts_alta, 'post_count', 4)

In [ ]:
# Aplicando a função
calcular_silhouette_e_inercia(post_counts_alta, kmeans, 'post_count')

In [ ]:
# Aplicando a função
calcular_inercia(post_counts_alta, 'post_count')

In [ ]:
# Aplicando a função
calcular_distancias_cluster(post_counts_alta, kmeans, 'post_count')

# Clusterizando DF Likes

In [ ]:
# Adicinando colunas no df_likes
df_likes['data'] = df_likes['likeCreatedTime'].dt.date
df_likes['data'] = pd.to_datetime(df_likes['data'])
df_likes['mes'] = df_likes['data'].dt.to_period('M')
df_likes['semana'] = df_likes['data'].dt.to_period('W')

df_likes_2017 = df_likes[df_likes['data'].dt.year == 2017]
df_likes_2017 = df_likes_2017[~((df_likes_2017['data'].dt.month == 12) & (df_likes_2017['data'].dt.year == 2017))]
df_likes_2017 = df_likes_2017[df_likes_2017['data'].dt.month >= 5]
df_likes_2017 = df_likes_2017

# Filtrar pelos likes dos id_usuarios que estão no df_posts
df_likes_2017 = df_likes_2017[df_likes_2017['id_usuario'].isin(df_posts['id_usuario'])]
df_likes_2017_normal = df_likes_2017[df_likes_2017['id_usuario'].isin(df_normal['id_usuario'])]
df_likes_2017_alta_ativ = df_likes_2017[df_likes_2017['id_usuario'].isin(df_alta_ativ['id_usuario'])]

In [ ]:
# Calcular a contagem de postagens por usuário
likes_counts = df_likes_2017.groupby('id_usuario').size().reset_index(name='like_count')

# Visualizar a distribuição
plotar_distribuicao(likes_counts, 'like_count', 'Likes', 'Distribuição de Likes por Usuário')

In [ ]:
df_likes_2017.to_csv('df_likes_2017.csv', index=False)
df_likes_2017.head()

In [ ]:
norm_post_count_like = df_likes_2017_normal.groupby('id_usuario').size().reset_index(name='like_count')
alt_post_count_like = df_likes_2017_alta_ativ.groupby('id_usuario').size().reset_index(name='like_count')

In [ ]:
plotar_distribuicao(norm_post_count_like, 'like_count', 'Likes', 'Distribuição de Likes por Usuário (Normal)')
plotar_distribuicao(alt_post_count_like, 'like_count', 'Likes', 'Distribuição de Likes por Usuário (Alta Atividade)')

# Usando o K-Means para agrupar likes

In [ ]:
# Criar cluster para os dados de likes
like_counts_normal = df_likes_2017_normal.groupby('id_usuario').size().reset_index(name='like_count')

# Visualizar a distribuição
plotar_distribuicao(like_counts_normal, 'like_count', 'Likes', 'Distribuição de Likes por Usuário')

In [ ]:
# Aplicando a função
kmeans, like_counts_normal = aplicar_kmeans(like_counts_normal, 'like_count', 3)

In [ ]:
# Aplicando a função
calcular_silhouette_e_inercia(like_counts_normal, kmeans, 'like_count')

In [ ]:
# Aplicando a função
calcular_inercia(like_counts_normal, 'like_count')

In [ ]:
# Aplicando a função
calcular_distancias_cluster(like_counts_normal, kmeans, 'like_count')

# Usuarios com alta atividade

In [ ]:
# Criar cluster para os dados de likes
like_counts_alta = df_likes_2017_alta_ativ.groupby('id_usuario').size().reset_index(name='like_count')

# Visualizar a distribuição
plotar_distribuicao(like_counts_alta, 'like_count', 'Likes', 'Distribuição de Likes por Usuário')

In [ ]:
# Aplicando a função
kmeans, like_counts_alta = aplicar_kmeans(like_counts_alta, 'like_count', 3)

In [ ]:
# Aplicando a função
calcular_silhouette_e_inercia(like_counts_alta, kmeans, 'like_count')

In [ ]:
# Aplicando a função
calcular_inercia(like_counts_alta, 'like_count')

In [ ]:
# Aplicando a função
calcular_distancias_cluster(like_counts_alta, kmeans, 'like_count')

# Analisar a Serie Temporal dos Cluster de Posts    

In [ ]:
def decompor_e_teste_mk(df_cluster, periodo, tipo_serie='posts'):
    resultados_mk = []

    for usuario in df_cluster['id_usuario'].unique():
        df_usuario = df_cluster[df_cluster['id_usuario'] == usuario]

        eventos_por_dia = df_usuario.groupby('data').size()

        if len(eventos_por_dia) >= 2 * periodo:
            try:
                resultado = seasonal_decompose(eventos_por_dia, model='additive', period=periodo)

                tendencia = resultado.trend.dropna()

                result = original_test(tendencia)

                resultados_mk.append({
                    'id_usuario': usuario,
                    'trend': result.trend,
                    'h': result.h,
                    'p-valor': result.p,
                    'z': result.z,
                    'Tau': result.Tau,
                    's': result.s,
                    'var_s': result.var_s,
                    'slope': result.slope,
                    'intercept': result.intercept
                })

            except ValueError as e:
                print(f"Erro ao decompor a série para o usuário {usuario}: {e}")
        else:
            print(
                f"Usuário {usuario} não tem observações suficientes para decomposição. Necessário: {2 * periodo}, disponível: {len(eventos_por_dia)}")

    df_resultados_mk = pd.DataFrame(resultados_mk)

    contagem_tendencias = df_resultados_mk['trend'].value_counts()
    print(contagem_tendencias)

    total_usuarios = df_resultados_mk.shape[0]
    percentuais = (contagem_tendencias / total_usuarios) * 100
    print(percentuais)

    # Gráfico de barras das tendências
    plt.figure(figsize=(8, 6))
    sns.countplot(data=df_resultados_mk, x='trend', order=['increasing', 'decreasing', 'no trend'])
    plt.title(f'Distribuição de Tendências entre Usuários ({tipo_serie.capitalize()})')
    plt.xlabel('Tipo de Tendência')
    plt.ylabel('Número de Usuários')
    plt.show()

    # Gráfico de distribuição dos slopes
    plt.figure(figsize=(10, 6))
    sns.histplot(df_resultados_mk['slope'], bins=20, kde=True)
    plt.title(f'Distribuição dos Slopes entre Usuários ({tipo_serie.capitalize()})')
    plt.xlabel('Slope')
    plt.ylabel('Frequência')
    plt.show()

    return df_resultados_mk

In [ ]:
# Criar um DataFrame para cada cluster de postagens
df_normal_cluster0 = df_normal[
    df_normal['id_usuario'].isin(post_counts_normal[post_counts_normal['cluster'] == 0]['id_usuario'])]

df_normal_cluster1 = df_normal[
    df_normal['id_usuario'].isin(post_counts_normal[post_counts_normal['cluster'] == 1]['id_usuario'])]

df_normal_cluster2 = df_normal[
    df_normal['id_usuario'].isin(post_counts_normal[post_counts_normal['cluster'] == 2]['id_usuario'])]

In [ ]:
# Criar um DataFrame para cada cluster de postagens
df_alta_ativ_cluster0 = df_alta_ativ[
    df_alta_ativ['id_usuario'].isin(post_counts_alta[post_counts_alta['cluster'] == 0]['id_usuario'])]

df_alta_ativ_cluster1 = df_alta_ativ[
    df_alta_ativ['id_usuario'].isin(post_counts_alta[post_counts_alta['cluster'] == 1]['id_usuario'])]

df_alta_ativ_cluster2 = df_alta_ativ[
    df_alta_ativ['id_usuario'].isin(post_counts_alta[post_counts_alta['cluster'] == 2]['id_usuario'])]

df_alta_ativ_cluster3 = df_alta_ativ[
    df_alta_ativ['id_usuario'].isin(post_counts_alta[post_counts_alta['cluster'] == 3]['id_usuario'])]

In [ ]:
# Aplicando a função
resultados_mk_normal_cluster0 = decompor_e_teste_mk(df_normal_cluster0, 30)

In [ ]:
# Aplicando a função
resultados_mk_normal_cluster1 = decompor_e_teste_mk(df_normal_cluster1, 30)

In [ ]:
# Aplicando a função
resultados_mk_normal_cluster2 = decompor_e_teste_mk(df_normal_cluster2, 30)

# Analisando a Tendência dos Clusters de usuários com alta atividade

In [ ]:
resultados_mk_alta_ativ_cluster0 = decompor_e_teste_mk(df_alta_ativ_cluster0, 30)

In [ ]:
resultados_mk_alta_ativ_cluster1 = decompor_e_teste_mk(df_alta_ativ_cluster1, 30)

In [ ]:
resultados_mk_alta_ativ_cluster2 = decompor_e_teste_mk(df_alta_ativ_cluster2, 30)

In [ ]:
resultados_mk_alta_ativ_cluster3 = decompor_e_teste_mk(df_alta_ativ_cluster3, 30)